### Продвинутые методы машинного обучения (1 семестр) 
### Индивидуальное задание по модулю «Визуализация данных»
#### **Выполнил:** Алексеев Арсалан Георгиевич

#### **Датасет №4:** Список персонажей DC Comics
https://www.kaggle.com/datasets/arunasivapragasam/dc-comics

#### Описание:
Мы запускаем свою линейку комиксов и нам нужно определить паттерн (набор черт, например: мужчина с голубыми глазами и с засекреченной личностью, но при этом злодей) пяти наиболее популярных (по числу появлений) персонажей.

#### Гипотеза:
Самые популярные персонажи часто похожи и можно из них выделить некие паттерны, на которые можно опираться при создании персонажей.

![DC comics image](https://assets.nationalnewswatch.com/wp-content/uploads/2013/12/superhero.jpg "DC comics image")

Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# python version 3.9.10

1. Проверим качество данных

In [2]:
# Считываем CSV file в pandas DataFrame
dc = pd.read_csv('data/dc-comics.csv')

In [3]:
# Ознакамливаемся с данными
dc.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,1237.0,"1940, April",1940.0


1.a Проверяем типы данных

In [11]:
# Смотрим общую информацию о данных
dc.info()

# Заметим, что только 3 первых столбца не имеют пропусков
# Тип данных столбца FIRST APPEARANCE - object, возможно нужно преобразование
# Остальные типы данных нормальные и в преобразовании не нуждаются

<class 'pandas.core.frame.DataFrame'>
Index: 6481 entries, 0 to 6531
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6481 non-null   object 
 1   ID           6481 non-null   object 
 2   ALIGN        6481 non-null   object 
 3   EYE          6481 non-null   object 
 4   HAIR         6481 non-null   object 
 5   SEX          6481 non-null   object 
 6   ALIVE        6481 non-null   object 
 7   APPEARANCES  6481 non-null   float64
 8   YEAR         6481 non-null   float64
dtypes: float64(2), object(7)
memory usage: 506.3+ KB


In [13]:
# Удалим неинформативные стобцы
dc.drop(columns=['page_id', 'urlslug', 'FIRST APPEARANCE'], inplace=True)

# Столбцы 'page_id', 'urlslug' не несут важную для исследования информацию
# Столбец 'FIRST APPEARANCE' дублирует год в столбце YEAR
# Оставим столбец YEAR, так как он уже находится в числовом формате

KeyError: "['page_id', 'urlslug', 'FIRST APPEARANCE'] not found in axis"

1.b Проверяем наличие пропусков

In [ ]:
display(dc.isnull().tail())
cols_null_percent = dc.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

# Изучим более подробно столбцы с пропусками,
# попытаемся понять с чем связаны пропуски

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
6891,False,False,False,True,True,False,False,True,True
6892,False,False,False,True,True,False,False,True,True
6893,False,False,False,True,True,False,False,True,True
6894,False,False,False,True,True,False,False,True,True
6895,False,False,False,False,False,False,False,True,True


EYE            52.610209
HAIR           32.975638
ID             29.190835
ALIGN           8.715197
APPEARANCES     5.147912
SEX             1.812645
YEAR            1.000580
ALIVE           0.043503
dtype: float64

In [ ]:
# Определим столбцы, наличие пропусков в которых может быть обосновано
dc.nunique()
cols_null_for_cat = dc[['ID', 'ALIGN', 'EYE', 'HAIR', 'SEX', 'ALIVE']]

# Посмотрим для каждого столбца уникальные значения

for column in cols_null_for_cat:
    dc_cols_unique_values = dc[column].unique()
    print(f"Column: {column}\nUnique Values: {dc_cols_unique_values}\n")

# Наличие пропусков в этих столбцах вполне возможно и объяснимо
# особенностями самого персонажа, а не браком в данных.

Column: ID
Unique Values: ['Secret Identity' 'Public Identity' nan 'Identity Unknown']

Column: ALIGN
Unique Values: ['Good Characters' 'Bad Characters' 'Neutral Characters' nan
 'Reformed Criminals']

Column: EYE
Unique Values: ['Blue Eyes' 'Brown Eyes' 'Green Eyes' 'Purple Eyes' 'Black Eyes'
 'White Eyes' 'Red Eyes' 'Photocellular Eyes' 'Hazel Eyes' 'Amber Eyes'
 'Yellow Eyes' nan 'Grey Eyes' 'Pink Eyes' 'Violet Eyes' 'Gold Eyes'
 'Orange Eyes' 'Auburn Hair']

Column: HAIR
Unique Values: ['Black Hair' 'Brown Hair' 'White Hair' 'Blond Hair' 'Red Hair' nan
 'Green Hair' 'Strawberry Blond Hair' 'Grey Hair' 'Silver Hair'
 'Orange Hair' 'Purple Hair' 'Gold Hair' 'Blue Hair' 'Reddish Brown Hair'
 'Pink Hair' 'Violet Hair' 'Platinum Blond Hair']

Column: SEX
Unique Values: ['Male Characters' 'Female Characters' nan 'Genderless Characters'
 'Transgender Characters']

Column: ALIVE
Unique Values: ['Living Characters' 'Deceased Characters' nan]



In [ ]:
# Заменим пропуски на 'Unknown' для удобства дальнейшей работы

for column in cols_null_for_cat:
    dc[column].fillna('Unknown', inplace=True)

# Заметим, что в столбце ID есть строки 'Identity Unknown', заменим их
dc['ID'] = dc['ID'].replace('Identity Unknown', 'Unknown')

# Удалим строки содержащие пропуски в столбцах 'APPEARANCES' и 'YEAR',
# Наличие информации в этих столбцах необходимо
dc.dropna(subset=['APPEARANCES', 'YEAR'], inplace=True)

#выводим результирующую долю пропусков
display(dc.isnull().mean())


name           0.0
ID             0.0
ALIGN          0.0
EYE            0.0
HAIR           0.0
SEX            0.0
ALIVE          0.0
APPEARANCES    0.0
YEAR           0.0
dtype: float64

1.c Проверяем наличие дубликатов

In [ ]:
dupl_columns = list(dc.columns)
mask = dc.duplicated(subset=dupl_columns)
dc_duplicates = dc[mask]
print(f'Число найденных дубликатов: {dc_duplicates.shape[0]}')
dc_dedupped = dc.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {dc_dedupped.shape[0]}')

# Дубликатов не обнаружено

Число найденных дубликатов: 0
Результирующее число записей: 6481


2.Рассчитаем основные показатели

In [ ]:
display(dc)

,name,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,YEAR
0,Batman (Bruce Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3093.0,1939.0
1,Superman (Clark Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2496.0,1986.0
2,Green Lantern (Hal Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,Living Characters,1565.0,1959.0
3,James Gordon (New Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,Living Characters,1316.0,1987.0
4,Richard Grayson (New Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,1237.0,1940.0
...,...,...,...,...,...,...,...,...,...
6527,Gypper Zilich (New Earth),Unknown,Unknown,Unknown,Black Hair,Male Characters,Living Characters,1.0,1941.0
6528,Robert Mason (New Earth),Secret Identity,Neutral Characters,Black Eyes,Black Hair,Male Characters,Living Characters,1.0,1941.0
6529,Diablo the Horse (New Earth),Unknown,Good Characters,Unknown,Black Hair,Male Characters,Living Characters,1.0,1940.0
6530,Scott Scanlon (New Earth),Public Identity,Unknown,Unknown,Blond Hair,Male Characters,Living Characters,1.0,1938.0
